## Imports

In [ ]:
# General downloads
!pip install transformers datasets
!pip install wptools
!pip install wikipedia 

import itertools 
import os 
import numpy 
import re 

# HuggingFace Transformers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline 
import tensorflow as tf
import spacy 

import nltk 
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

# MediaWiki API 
import wptools 
import wikipedia 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.metrics.pairwise import linear_kernel 

In [ ]:
# Model trained on the SQuAD 2.0 dev set 
model_name = "deepset/roberta-base-squad2"

qa_final = pipeline('question-answering', model = model_name, tokenizer = model_name) 

## Modules

In [ ]:
def getKeywords(question): 
    tagged = nltk.pos_tag(nltk.word_tokenize(question)) 
    print(tagged) 

    # The NLTK POS Tagger follows the Penn Treebank Project tag conventions 
    # Only the following kinds of words are extracted from the query as keywords 
    limit = ['FW', 'JJ', 'JJS', 'JJR', 'NN', 'NNS', 'NNP', 'NNPS', 'SYM'] 
    keywords = ' '.join(i[0] for i in tagged if i[1] in limit) 
    print(keywords) 
    return keywords 

In [ ]:
def retrieveDocs(keywords): 
    wiki_search = wikipedia.search(keywords) 
    print("Wiki search results:") 
    print(wiki_search) 
    print("\n" + '='*(20) + "\n")
    documents = []
    documentTitles = []
    for i in wiki_search: 
        page = wptools.page(str(i))
        page.get_parse() 
        # print(str(i), page.data['pageid']) 
        try: 
            content = wikipedia.page(pageid=page.data['pageid']) 
        except: 
            continue
        documentTitles.append(str(i))
        res = cleanDoc(content.content)
        documents.append(res) 
    print("Entries considered:") 
    print(documentTitles) 
    print("\n" + '='*(20) + "\n")
    return documents 

In [ ]:
def cleanDoc(content):
      headings_to_remove = ['== Further reading ==', '== Further references ==', '=== Citations ===', '== References ==', '== Footnotes ==', 
                            '=== Notes ===', '== Notes ==', '=== Sources ===', '== Sources ==', '== External links', '== See also ==', ]
      headings_to_remove = '|'.join(headings_to_remove) 
      inds = [m.start() for m in re.finditer(headings_to_remove, content)]
      # print(inds) 
      if len(inds) != 0: 
          mini = min(inds) 
          mini = min(mini, len(content)) 
      else: 
          mini = len(content)
      # print(mini)
      return content[:mini]

In [ ]:
def splitDocs(question, documents): 
    passages = [question]
    for i in documents: 
        curr_passages = [p for p in i.split('\n') if p and not p.startswith('=')] 
        passages += curr_passages 
    return passages 

In [ ]:
# def retrievePassages(passages): 
#     tfidf = TfidfVectorizer().fit_transform(passages) 
#     cosSims = linear_kernel(tfidf[0:1], tfidf).flatten()
#     # print(cosSims) 
#     passageInds = cosSims.argsort()[:-12:-1]
#     print("Indices of most relevant passages: ")
#     print(passageInds[1:]) 
#     print("\n" + '='*(20) + "\n")
#     return passageInds 

# def printRelevantPassages(passages, passageInds): 
#     print("Most relevant passages: ")
#     for i in range(1, len(passageInds)): 
#         print(passages[passageInds[i]]) 
#     print("\n" + '='*(20) + "\n") 

# def getAnswers(passages, passageInds): 
#     possibleAnswers = []
#     for i in range(1, len(passageInds)): 
#         possibleAnswers.append(qa_final(question = passages[0], context = passages[passageInds[i]])) 
#     # print(possibleAnswers) 
#     possibleAnswers = sorted(possibleAnswers, key = lambda i: i['score']) 
#     return possibleAnswers 

In [ ]:
# def retrievePassages(question, documents): 
#     passages = {}
#     for i in documents: 
#         curr_passages = [p for p in i.split('\n') if p and not p.startswith('=')] 
#         curr_passages.insert(0, question) 
#         tfidf = TfidfVectorizer().fit_transform(curr_passages) 
#         cosSims = linear_kernel(tfidf[0:1], tfidf).flatten()
#         print(cosSims) 
#         passageInds = cosSims.argsort()[:-12:-1] 
#         print(cosSims) 
#         print(passageInds)
#         for i in range(1, len(passageInds)): 
#             passages[curr_passages[passageInds[i]]] = cosSims[i]
#         # passages.append(curr_passages[passageInds[-1]]) 
#     return passages 

def retrievePassages(question, documents): 
    passages = {}
    for i in documents: 
        curr_passages = [p for p in i.split('\n') if p and not p.startswith('=')] 
        curr_passages.insert(0, question) 
        tfidf = TfidfVectorizer().fit_transform(curr_passages) 
        cosSims = linear_kernel(tfidf[0:1], tfidf).flatten() 
        curr_passages = dict(zip(curr_passages, cosSims)) 
        curr_passages = dict(sorted(curr_passages.items(), key = lambda item: item[1], reverse = True)) 
        p = list(curr_passages.keys()) 
        s = list(curr_passages.values()) 
        i = 1 
        while i < len(p) and i < 10: 
          passages[p[i]] = s[i] 
          i += 1

    print(passages) 
    passages = dict(sorted(passages.items(), key = lambda item: item[1], reverse = True)) 
    print(passages.values())
    passages = list(passages.keys())[:10] 
    print(passages) 
    return passages 

In [ ]:
def printRelevantPassages(passages): 
    print("Most relevant passages: ")
    for i in passages: 
        print(i) 
    print("\n" + '='*(20) + "\n") 

In [ ]:
def getAnswers(question, passages): 
    possibleAnswers = []
    for i in passages: 
        possibleAnswers.append(qa_final(question = question, context = i)) 
    # print(possibleAnswers) 
    possibleAnswers = sorted(possibleAnswers, key = lambda i: i['score']) 
    return possibleAnswers 

In [ ]:
def printAllAnswers(possibleAnswers): 
    print("Possible answers sorted by confidence rating: ")
    for i in range(len(possibleAnswers) - 1, -1, -1): 
        print(str(len(possibleAnswers) - 1 - i + 1) + '.' + possibleAnswers[i]['answer'] + ':' + str(possibleAnswers[i]['score'])) 
    print("\n" + '='*(20) + "\n") 

## System

In [ ]:

##### 
# Type either keywords only or the entire question itself 
# Does not yet work for yes/no questions like "Is Australia a Continent?" 
# Model used to derive answers from context will be modified to increase accuracy, and NLG for the answer will also be tried out 
# Example questions that work: What is the capital of Assam?, Who is the Greek goddess of Wisdom?, Where is Addis Ababa? 
# Example questions that don't work: Who played Harley Quinn in the Suicide Squad?, What is a binary search tree? Who is the CEO of Apple? 
#####

question = input("Enter question: ") 

In [ ]:
# keywords = getKeywords(question) 
# documents = retrieveDocs(keywords) 
# passages = splitDocs(question, documents) 
# passageInds = retrievePassages(passages) 
# printRelevantPassages(passages, passageInds) 
# possibleAnswers = getAnswers(passages, passageInds) 
# printAllAnswers(possibleAnswers) 

In [ ]:
keywords = getKeywords(question) 
documents = retrieveDocs(keywords) 
passages = retrievePassages(question, documents) 
printRelevantPassages(passages) 
possibleAnswers = getAnswers(question, passages) 
printAllAnswers(possibleAnswers) 

In [ ]:
print(question) 
print(possibleAnswers[-1]['answer'])